<a href="https://colab.research.google.com/github/akhil9900/Homeassignment5/blob/main/home_assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#question 3

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import os

# Set up
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
latent_dim = 100
image_size = 28*28

# Generator
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, image_size),
            nn.Tanh()
        )
    def forward(self, z):
        return self.model(z)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(image_size, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)

G = Generator().to(device)
D = Discriminator().to(device)

# Data loader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_loader = DataLoader(
    datasets.MNIST('.', download=True, transform=transform),
    batch_size=64, shuffle=True
)

criterion = nn.BCELoss()
optimizer_G = optim.Adam(G.parameters(), lr=0.0002)
optimizer_D = optim.Adam(D.parameters(), lr=0.0002)

# Training loop
os.makedirs("generated_images", exist_ok=True)
for epoch in range(101):
    for batch, (real_imgs, _) in enumerate(train_loader):
        real_imgs = real_imgs.view(-1, 28*28).to(device)
        batch_size = real_imgs.size(0)

        # Real and fake labels
        real = torch.ones(batch_size, 1).to(device)
        fake = torch.zeros(batch_size, 1).to(device)

        # Train Discriminator
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_imgs = G(z)
        loss_D = criterion(D(real_imgs), real) + criterion(D(fake_imgs.detach()), fake)
        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

        # Train Generator
        z = torch.randn(batch_size, latent_dim).to(device)
        loss_G = criterion(D(G(z)), real)
        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

    if epoch % 50 == 0 or epoch == 0:
        print(f"Epoch {epoch}, Loss D: {loss_D.item()}, Loss G: {loss_G.item()}")
        save_image(G(z).view(-1, 1, 28, 28), f"generated_images/epoch_{epoch}.png", normalize=True)


100%|██████████| 9.91M/9.91M [00:01<00:00, 5.97MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 159kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.50MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.19MB/s]


Epoch 0, Loss D: 0.9679704904556274, Loss G: 1.0020819902420044
Epoch 50, Loss D: 0.8153786659240723, Loss G: 1.4780302047729492
Epoch 100, Loss D: 0.48325109481811523, Loss G: 2.940903663635254
